<a href="https://colab.research.google.com/github/KyriakosPsa/Bayesian-linear-regression/blob/master/Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive

import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessing

In [26]:
data = pd.read_csv(
    "/content/drive/MyDrive/Datasets/GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt", delimiter="\t")
print("The shape of the dataframe is: ",data.shape)
data.head()

The shape of the dataframe is:  (57915, 566)


,gene_id,gene_name,gene_type,BC01_Pooled,BC01_Tumor,BC02_Pooled,BC03_Pooled,BC03LN_Pooled,BC04_Pooled,BC05_Pooled,...,BC11_04,BC11_07,BC11_28,BC11_43,BC11_56,BC11_69,BC11_70,BC11_78,BC11_81,BC11_88
0,ENSG00000000003.10,TSPAN6,protein_coding,2.33,1.25,43.96,7.64,9.32,133.37,84.77,...,0.00,47.44,16.11,0.00,6.56,0.00,0.00,0.00,200.82,0.00
1,ENSG00000000005.5,TNMD,protein_coding,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ENSG00000000419.8,DPM1,protein_coding,60.70,28.44,74.73,41.41,87.69,120.41,70.67,...,198.96,0.00,0.00,20.63,0.00,0.00,0.00,136.07,22.03,204.58
3,ENSG00000000457.9,SCYL3,protein_coding,47.93,4.43,9.89,7.61,7.32,12.42,12.02,...,0.00,10.96,0.00,0.00,0.00,0.27,0.00,0.07,0.19,9.52
4,ENSG00000000460.12,C1orf112,protein_coding,4.79,1.67,10.87,0.92,15.13,17.10,5.69,...,0.00,6.97,9.02,0.00,0.00,0.14,247.69,22.60,11.35,0.00


This is not the usual shape of a single cell counts so we transform it accordingly

In [45]:
counts = pd.concat([data.iloc[:,1], data.iloc[:,3:]],axis = 1)
counts.set_index('gene_name',inplace = True) 
counts = counts.T
cells = counts.index
genes = counts.columns
gene_type = data

array([[2.3300000e+00, 0.0000000e+00, 6.0700000e+01, ..., 1.4940700e+04,
        9.8582000e+02, 0.0000000e+00],
       [1.2500000e+00, 0.0000000e+00, 2.8440000e+01, ..., 4.5728190e+04,
        6.0750000e+02, 0.0000000e+00],
       [4.3960000e+01, 0.0000000e+00, 7.4730000e+01, ..., 1.0442910e+04,
        3.6255000e+02, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 1.3607000e+02, ..., 3.3465110e+04,
        1.6521900e+03, 0.0000000e+00],
       [2.0082000e+02, 0.0000000e+00, 2.2030000e+01, ..., 1.0890135e+05,
        3.4395300e+03, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 2.0458000e+02, ..., 5.7492010e+04,
        1.8374100e+03, 0.0000000e+00]])

Now we can load it in a sparse matrix and into an adata object

In [46]:
counts_tmp = csr_matrix(counts.values, dtype=np.float32)
adata = ad.AnnData(counts_tmp)

<563x57915 sparse matrix of type '<class 'numpy.float32'>'
	with 3312222 stored elements in Compressed Sparse Row format>

In [47]:
adata.obs_names = cells
adata.var_names = genes 

In [ ]:
adata.obs["cell_type"]

In [51]:
print(adata.obs_names[:10])

Index(['BC01_Pooled', 'BC01_Tumor', 'BC02_Pooled', 'BC03_Pooled',
       'BC03LN_Pooled', 'BC04_Pooled', 'BC05_Pooled', 'BC06_Pooled',
       'BC07_Tumor', 'BC07LN_Pooled'],
      dtype='object')
